# Retrieving Arrival Time from Real Time Data

## Setting Up Environment

In this part, we will set up the environment.
- Installing google cloud environtment
- Installing necessary libraries
- Importing necessary libraries
- Define shared variables and functions

<br/>

### Saving file via VM environment
**Important notes: We use gcsfuse to combine google cloud storage and google collab directories, but there's permission issue when saving a file in vm instance**

- Upload credential file to content directory in google collab
- enable VM environment settings
- replace this syntax<br />From this:
```
   to_csv('content/main/path')
```   
To this:
```
  to_csv(f'gs://{bucket.name}/path')
```


In [ ]:
'''
  Initiate Auth to access google cloud storage & setup necessary package
'''

# Disable in vm, enable in colab local
# from google.colab import auth
# auth.authenticate_user()

# Disable in colab environment
from google.cloud import storage
client = storage.Client.from_service_account_json("/content/fresh-tape-370916-d7d7c2e7c5c4.json")
bucket = client.get_bucket('datamining-ulb')

# Enable in collab runtime
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse
!mkdir main
!gcsfuse --implicit-dirs datamining-ulb main

!ls main

!pip install jenkspy
!pip install ckwrap
!pip install dbscan1d
!pip install fsspec
!pip install gcsfs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2426  100  2426    0     0  65567      0 --:--:-- --:--:-- --:--:-- 65567
OK
W: GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
E: The repository 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease' is no longer signed.
gcsfuse is already the newest version (0.41.9).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 315 not upgraded.
mkdir: cannot create directory ‘main’: File exists
2022/12/18 16:11:46.840036 Start gcsfuse/0.41.9 (Go version go1.18.4) for app "" using mount point: /content/main
2022/

In [ ]:
'''
  Import common library
'''

import pandas as pd
import re
import numpy as np
import math
import datetime
import decimal
import os

from glob import glob
from dateutil import parser
import csv
from io import StringIO

In [ ]:
'''
 Helper functions & constants
'''

decimal.getcontext().prec = 3
TO_MS = decimal.Decimal("1000")
TWO_HOURS = 7200000
STOP_DISTANCE_THRESHOLD = 5 # In metres
AVERAGE_SPEED = {"bus": decimal.Decimal("4.305"), "tram": decimal.Decimal("4.5"), "metro": decimal.Decimal("7.75")}  # in metre/second

'''
Convert epoch milisecond to datetime format:
 i.e 165278383333 -> 2022-12-01 12:23:20
'''
def epoch_ms_to_datetime(epoch_ms: int) -> datetime.datetime:
  return datetime.datetime.fromtimestamp(epoch_ms / int(TO_MS))

'''
  Convert datetime to epoch:
  i.e 2022-12-01 12:23:20 -> 165278383333
'''
def datetime_to_epoch(datetime: datetime) -> int:
  return datetime.timestamp()

def save_csv(path, df):
  """for saving csv files"""
  save_df = df.to_csv(sep=",", index=False, quotechar='"', quoting=csv.QUOTE_ALL, encoding="UTF-8")
  blob = bucket.blob(path)
  blob.upload_from_string(data=save_df)

def save_np_csv(path,np_arr):
  """for saving numpy arrays to csv"""
  s = StringIO()
  save_np = np.savetxt(s, np_arr, delimiter=",", fmt='%s')
  blob = bucket.blob(path)
  blob.upload_from_string(data=s.getvalue())

def get_column_idx(column_label, df):
  return [idx for idx, col in enumerate(df.columns) if col==column_label][0]

def get_real_date(year, month, day, hour, minute, second):
  date = datetime.datetime(year,month,day)
  if hour > 23:
    date += datetime.timedelta(days=1)
    hour = hour % 24
  date += datetime.timedelta(hours=hour, minutes=minute, seconds=second)
  return date

# Cleaning The Data

We follow several steps to clean the data:

## Marking Technical Stops

- We extract technical stops from the shapefile data. We loaded the data using postgis and export it to csv files to retrieve it in python
- After that put the orders of the stops as new parameter by matching the pointID and directionId from the ac. The orders is only used to verify the stops and helps us to visualize the data.
- We also put mark into the stops if we cannot find the stops id or the direction id from the actual stops data. We will mark this pointID as techincal stop

## Segments Data

- We will separated the data by its type (bus, tram, metro) and also identify whether the line has abnormal distance (contains only 0 and 1) or has normal distance.

## Distance Normalization

### Abnormal Distance
After identfying the abnormal and normal distance, we will normalize the distance using this algorithm:

1) For each timestamp, each direction, each pointID, count how many 1s
2) say there are N, then we get distancr from pointID to the next one in the Sequence, csll this distance D
3) divide D/(N+1)=p
4) set distances to be p, 2p, ..., Np

### Normalize Non-Zero Distance
We will convert non-zero distance, and calculate estimated timestamp for -5 & +5 distanceFromPoint.

1. Pick non zero distance and interpolate it by doing estimated timestamp calculation for -5 & +5 distance.
2. We will then sort the data based on the estimated timestamp
3. If we find consecutive non-zero data with the same distance, we will remove it to declutter the data.

### Grouping the Data
We will group the result by line id and point id and separate it into different files. It will make the clustering and analyzing result much more easier since it is already grouped per stop id.


In [ ]:
'''
  Filter out technical stops or non existing stops from the real data
'''

# Extract actual stops
actual_df = pd.read_csv("/content/main/data/actual_stops.csv")
size = actual_df.shape[0]
actual_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in actual_df["stop_id"]]

actual_df["succession"] = actual_df["succession"].astype(str)
actual_df["numero_lig"] = actual_df["numero_lig"].astype(str)
actual_df["variante"] = actual_df["variante"].astype(str)
actual_stops = set(actual_df["stop_id"])

# Extract Line stop sequence
extracted_lines = {}

delimiters = np.array(["_" for _ in range(size)])
stops_info = (actual_df["numero_lig"].to_numpy() + delimiters +  actual_df["succession"].to_numpy() + delimiters + actual_df["variante"].to_numpy() + delimiters + actual_df["stop_id"].to_numpy()).tolist()

prev_succession = -999999
direction = ""
for stops in stops_info[::-1]:
    info = stops.split("_")
    if int(prev_succession) < int(info[1]):
        direction = info[3]

    data = extracted_lines.setdefault(info[0], {})

    succession_info = data.setdefault(info[3], {})
    succession_info[direction] = info[1]

    data[info[3]] = succession_info
    extracted_lines[info[0]] = data

    line_direction = data.setdefault("direction", {})
    visited_stops = line_direction.setdefault(direction, set())
    visited_stops.add(info[3])

    line_direction[direction] = visited_stops
    data["direction"] = line_direction

    prev_succession = info[1]

'''
Pair stops with the succession line,
 1. if there's multiple sucession line check whether direction is the same
 2. If not, try to find direction id in the direction stops
 3. otherwise set sucession to 99999
'''

lines = []
names = set()
for file in glob("/content/main/ordered_lines/*.csv"):
  name = file.strip().split("/")[4].split("_")[0]
  names.add(name)

print(names)
for file in glob("/content/main/Lines_vehiclePositions/*.csv"):
    if "null" in file or file in names:
      continue

    df = pd.read_csv(file)
    df["lineID"] = df["lineID"].astype(str)
    df["directionID"] = df["directionID"].astype(str)
    df["pointID"] = df["pointID"].astype(str).apply(lambda x: x.strip().split(" ")[0])

    delimiters = np.array(["_" for _ in range(df.shape[0])])
    stops_infos = (df["lineID"].to_numpy() + delimiters + df["directionID"].to_numpy() + delimiters + df["pointID"].to_numpy() + delimiters + df.index.astype(str).values).tolist()

    succession_arr = []
    for stop in stops_infos:
        info = stop.split("_")
        data = extracted_lines[info[0]]

        succession = 999999
        if info[2] not in data:
            succession_arr.append(succession)
            continue

        succession_info = data[info[2]]
        if info[1] not in succession_info:
            succession_arr.append(succession)
            continue

        succession = succession_info[info[1]]
        succession_arr.append(succession)

    df["succession"] = np.array(succession_arr)

    df["lineID"] = df["lineID"].astype(int)
    df["directionID"] = df["directionID"].astype(int)
    df["pointID"] = df["pointID"].astype(int)
    df["succession"] = df["succession"].astype(int)

    name = file.strip().split("/")[4].split("_")[0]
    df.sort_values(["directionID", "time",  "succession"], ascending=[True, True, True]).to_csv(f"/content/main/ordered_lines/{name}_ordered.csv", index=False)


{'Line71', 'Line93', 'Line20', 'Line78', 'Line6', 'Line28', 'Line95', 'Line33', 'Line27', 'Line38', 'Line47', 'Line79', 'Line13', 'Line62', 'Line48', 'Line43', 'Line58', 'Line92', 'Line46', 'Line12', 'Line42', 'Line57', 'Line74', 'Line1', 'Line88', 'Line89', 'Line77', 'Line54', 'Line19', 'Line39', 'Line17', 'Line56', 'Line69', 'Line72', 'Line81', 'Line61', 'Line8', 'Line80', 'Line36', 'Line50', 'Line76', 'Line5', 'Line7', 'Line70', 'Line9', 'Line21', 'Line60', 'Line63', 'Line64', 'Line75', 'Line14', 'Line98', 'Line41', 'Line45', 'Line66', 'Line53', 'Line59', 'Line51', 'Line55', 'Line82', 'Line44', 'Line37', 'Line2', 'Line29', 'Line34', 'Line49', 'Line86', 'Line25', 'Line65', 'Line4', 'Line97', 'Line3', 'Line87', 'Line83'}


In [ ]:
'''
Marking technical stops
'''
# Extract all stops
# using stops.txt as it has all the stops
actual_df = pd.read_csv("/content/main/schedule/gtfs23Sept/stops.csv")
actual_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in actual_df["stop_id"]]
actual_stops = set(actual_df["stop_id"])

for filename in glob("/content/main/ordered_lines/*.csv"):
# filename = "/content/main/ordered_lines/Line25_ordered.csv"
  if "null" in filename:
    continue

  cur_lineID = str(int(filename.split('/')[-1].split('_')[0][4:]))
  df = pd.read_csv(filename)
  df["pointID"] = df["pointID"].astype(str).apply(lambda x: x.strip().split(" ")[0])

  df['technical'] = False
  for idx, row in df.iterrows():
    if (str(row['pointID']) not in actual_stops) or (str(row['directionID']) not in actual_stops):
      df.at[idx, 'technical'] = True

  df.sort_values(["directionID", "time",  "succession",], ascending=[True, True, True]).to_csv(f"/content/main/ordered_lines/Line{cur_lineID}_ordered.csv", index=False)
  print(f"L{cur_lineID}: {df.loc[df['technical'] == True].shape[0]}")

L25: 127286


In [ ]:
# removing trailing 0s from stop_ids
actual_filename = "/content/main/data/actual_stops.csv"
actual_df = pd.read_csv(actual_filename)
# actual_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in actual_df["stop_id"]]
actual_df["stop_id"] = actual_df["stop_id"].apply(lambda x: x.lstrip("0"))
actual_df.to_csv(actual_filename, index=False)

In [ ]:
# removing unnecessary indeces
for filename in glob("/content/main/ordered_lines/*.csv"):
  if "null" in filename:
    continue

  df = pd.read_csv(filename)
  cur_lineID = str(int(filename.split('/')[-1].split('_')[0][4:]))

  for column in df.columns:
    if 'Unnamed' in column:
      df = df.drop([column], axis=1)

  df.sort_values(["directionID", "time",  "succession",], ascending=[True, True, True]).to_csv(f"/content/main/ordered_lines/Line{cur_lineID}_ordered.csv", index=False)

In [ ]:
'''
    Analyze which is the tram, bus or metro. Also, analyze whether the stops is only consist of 0 or 1 distance
'''
# Manually set the data from bus, tram, or metro based on stib website
metro = {"1", "2", "5", "6"}
tram = {"3", "4", "7", "8", "9", "19", "25", "39", "44", "51", "55", "62", "81", "82", "92", "93", "97"}

# Iterate to stops directory
for filename in glob("/content/main/ordered_lines/*.csv"):
    df = pd.read_csv(filename)

    line = str(df["lineID"][0])
    line_type =  "metro" if line in metro else "tram" if line in tram  else "bus"
    is_zero_one = all(distance == 0 or distance == 1 for distance in df["distancefromPoint"].astype(int).to_numpy())
    folder_name = {True: "AbnormalDistance", False: "NormalDistance"}

    name = filename.strip().split("/")[4].split("_")[0]
    df.to_csv(f"/content/main/ordered_lines/{folder_name[is_zero_one]}/{line_type}/{name}_ordered.csv")

In [ ]:
# Analyzing metro (0/1) lines
filename = "/content/main/ordered_lines/Line1_ordered.csv"
df = pd.read_csv(filename)
df = df.loc[df['technical']==False]
df["lineID"] = df["lineID"].astype(str)
df["directionID"] = df["directionID"].astype(str)
df["pointID"] = df["pointID"].astype(str).apply(lambda x: x.strip().split(" ")[0])
cur_lineID = str(int(filename.split('/')[-1].split('_')[0][4:]))

ones_df = df.loc[df['distancefromPoint']==1]

In [ ]:
filename = "/content/main/data/distance-between-points.csv"
distances_df = pd.read_csv(filename)

# detailed but does not contain all stops
actual_df = pd.read_csv("/content/main/data/actual_stops.csv")
actual_df["succession"] = actual_df["succession"].astype(str)
actual_df["numero_lig"] = actual_df["numero_lig"].astype(str)
actual_df["variante"] = actual_df["variante"].astype(str)
actual_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in actual_df["stop_id"]]

# contains all stops but not as detailed
stops_df = pd.read_csv("/content/main/schedule/gtfs23Sept/stops.csv")
stops_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in stops_df["stop_id"]]

In [ ]:
def get_next_stop(cur_pointID, cur_dirID, cur_lineID):
  '''
  gets decriptons of current and next bus stops,
  takes into consideration the directionID (cur_dirID), lineID (cur_lineID) and this stop's pointID (cur_pointID)
  accounts for cases when dirID is not the terminal, but an earlier stop

  requires stops.csv set as stops_df, actual_stops.csv as actual_df
  '''
  dirID_info = stops_df.loc[stops_df['stop_id'] == cur_dirID]
  if dirID_info.shape[0] == 0:
    print(f'No such stop: {cur_dirID}')
  dirID_desc = dirID_info.iloc[0]['stop_name']

  pointID_info = stops_df.loc[stops_df['stop_id'] == cur_pointID]
  if pointID_info.shape[0] == 0:
    print(f'No such stop: {cur_pointID}')
  pointID_desc = pointID_info.iloc[0]['stop_name']

  cur_pointID_info = actual_df.loc[(actual_df['descr_fr']==pointID_desc) & (actual_df['numero_lig'] == cur_lineID) & (actual_df['terminus']==dirID_desc)]
  if cur_pointID_info.shape[0] == 0:
    # if the directionId is not the actual terminal but an earlier stop
    # find the actual terminal from the "direcitionID" information
    cur_dirID_info = actual_df.loc[(actual_df['descr_fr']==dirID_desc) & (actual_df['numero_lig'] == cur_lineID) & (actual_df['stop_id']==cur_dirID)]
    if cur_dirID_info.shape[0] != 1:
      print(f'Not 1 match for cur_dirID {cur_dirID}, descr {dirID_desc}')

      # todo: if the stop_id of "directionID" is not of this line, maybe the description of the bus stop is correct
      # find the combination of this line where current pointID succession is smaller than "directionID" succession

    actual_terminal = cur_dirID_info.iloc[0]['terminus']

    cur_pointID_info = actual_df.loc[(actual_df['descr_fr']==pointID_desc) & (actual_df['numero_lig'] == cur_lineID) & (actual_df['terminus']==actual_terminal)]

  elif cur_pointID_info.shape[0] != 1:
    print(f'Not 1 match for cur_pointID {pointID_desc} terminus {actual_terminal}')

  else:
    actual_terminal = dirID_desc

  cur_succession = cur_pointID_info.iloc[0]['succession']

  # find the pointId of the next succession
  next_succession = str(int(cur_succession)+1)
  next_pointID_info = actual_df.loc[(actual_df['numero_lig'] == cur_lineID) & (actual_df['terminus']==actual_terminal) & (actual_df['succession']==next_succession)]

  if next_pointID_info.shape[0] != 1:
    print(f'Not 1 match for next_pointID for terminus: {actual_terminal}, succession {succession}')

  dirID_desc = dirID_info.iloc[0]['stop_name']

  return pointID_desc, next_pointID_info.iloc[0]['descr_fr']

In [ ]:
'''
Estimating distances for 0/1 lines (metros)
'''
for filename in glob("/content/main/ordered_lines/AbnormalDistance/*/*.csv"):
  cur_lineID = str(int(filename.split('/')[-1].split('_')[0][4:]))
  print('line', cur_lineID)
  df = pd.read_csv(filename)
  df = df.loc[df['technical']==False]
  df["lineID"] = df["lineID"].astype(str)
  df["directionID"] = df["directionID"].astype(str)
  df["pointID"] = df["pointID"].astype(str).apply(lambda x: x.strip().split(" ")[0])

  ones_df = df.loc[df['distancefromPoint']==1]

  for cur_time in ones_df.time.unique():
      # for each timestamp
      cur_time_set = ones_df.loc[ones_df['time']==cur_time]

      for cur_pointID in cur_time_set.pointID.unique():
          # for each bus_stop in that timestamp
          cur_pointID_set = cur_time_set.loc[cur_time_set['pointID']==cur_pointID]

          for cur_dirID in cur_pointID_set.directionID.unique():
              target_set = cur_pointID_set.loc[cur_pointID_set['directionID']==cur_dirID]
              # print(cur_time, cur_pointID, cur_dirID)
              N = len(target_set)
              try:
                  pointID_desc, next_pointID_descr = get_next_stop(cur_pointID, cur_dirID, cur_lineID)
                  D = distances_df.loc[(distances_df['from_descr_fr']==pointID_desc) & (distances_df['to_descr_fr']==next_pointID_descr)].iloc[0]['dist']
                  p = D/(N+1)
                  i = 1
                  for idx, row in target_set.iterrows():
                      ones_df.at[idx,'distancefromPoint']=i*p
                      i+=1
              except Exception as e:
                for idx, row in target_set.iterrows():
                    ones_df.at[idx,'distancefromPoint']=999999

  # saving to the actual df
  df.loc[df.index.isin(ones_df.index), ['distancefromPoint']] = ones_df[['distancefromPoint']]
  df.sort_values(["directionID", "time",  "pointID",], ascending=[True, True, True]).to_csv(f"/content/main/ordered_lines/Line{cur_lineID}_1_ordered.csv", index=False)

In [ ]:
'''
Normalize the distance
 The algorithm will work as follow:
 1. We will change the distance for non-zero distance point by subtracting the time with how much time passed since the bus in the 0 distance of the point
 2. We will use the bus/tram average speed to calculate the distance passed
'''

# Only process normal line
for filename in glob("/content/main/ordered_lines/NormalDistance/tram/*.csv"):
    line_df = pd.read_csv(filename)
    line_df = line_df[(line_df["technical"] == False) & (line_df["distancefromPoint"]) != 999999]
    line_df = line_df.sort_values(["directionID", "pointID", "time"])

    line_id = filename.split("/")[-1].split("_")[0]
    line_type = filename.split("/")[5].strip()

    columns = ["time", "lineID", "directionID", "distancefromPoint", "pointID", "succession"]

    size = line_df.shape[0]
    delimiters = np.array(["_" for _ in range(size)])

    line_infos = None
    for idx, column in enumerate(columns):
        line_df[column] = line_df[column].astype(str)
        if line_infos is None:
            line_infos = line_df[column].to_numpy() + delimiters
        elif line_infos is not None and idx < len(columns)-1:
            line_infos += line_df[column].to_numpy() + delimiters
        else:
            line_infos += line_df[column].to_numpy()


    # Array for building new dataset
    converted_timestamp = []
    converted_distances = []
    direction_ids = []
    point_ids = []
    line_ids = []
    orders = []
    original_timestamp = []
    original_distances = []

    for idx, line_info in enumerate(line_infos):
        line = line_info.split("_")

        # Unpack information
        timestamp = int(line[0])
        line_id = line[1]
        direction_id = line[2]
        distance = decimal.Decimal(line[3])
        point_id = line[4]
        order = int(line[5])

        # Ignore stopping bus
        if int(distance) == 0:
            line_ids.append(line_id)
            direction_ids.append(direction_id)
            point_ids.append(point_id)
            orders.append(order)

            original_timestamp.append(epoch_ms_to_datetime(timestamp))
            converted_timestamp.append(epoch_ms_to_datetime(timestamp))
            original_distances.append(distance)
            converted_distances.append(distance)
            continue

        # Calculate -5 meters and +5 meters from stop
        # The interpolation threshold is chosen by using heuristic method
        # We will consider bus position -5 metres or +5 metres from stop is considered as stopping
        # Normalized the timestamp
        timestamp_plus = timestamp - (int(((distance+STOP_DISTANCE_THRESHOLD) / AVERAGE_SPEED[line_type])*TO_MS))
        timestamp_min = timestamp - (int(((distance-STOP_DISTANCE_THRESHOLD) / AVERAGE_SPEED[line_type])*TO_MS))

        timestamps = [timestamp_min, timestamp_plus]
        time_passed = [(int((distance-STOP_DISTANCE_THRESHOLD) / AVERAGE_SPEED[line_type])), (int((distance+STOP_DISTANCE_THRESHOLD) / AVERAGE_SPEED[line_type]))]

        # Add the interpolation data
        for idx in range(len(timestamps)):
            line_ids.append(line_id)
            direction_ids.append(direction_id)
            point_ids.append(point_id)
            orders.append(order)

            original_timestamp.append(epoch_ms_to_datetime(timestamp))
            converted_timestamp.append(epoch_ms_to_datetime(timestamps[idx]))

            original_distances.append(distance)
            converted_distances.append(distance - (AVERAGE_SPEED[line_type]*time_passed[idx]))

    columns = {
                "line_id": np.array(line_ids),
               "direction_id": np.array(direction_ids),
               "point_id": np.array(point_ids),
               "order": np.array(orders),
               "original_timestamp": np.array(original_timestamp),
               "converted_timestamp": np.array(converted_timestamp),
               "original_distance": np.array(original_distances),
               "converted_distance": np.array(converted_distances)
              }


    converted_df = pd.DataFrame(columns).sort_values(["direction_id", "order", "converted_timestamp"], ascending=[True, True, True]).reset_index(drop=True)

    distances = converted_df['original_distance'].to_numpy()
    points = converted_df['point_id'].to_numpy()
    removed_idxs = []

    # Remove consecutive distance
    for idx in range(len(distances)):
      if distances[idx] != 0 and idx < len(distances) - 2 and int(distances[idx]) == int(distances[idx+2]) and str(points[idx]) == str(points[idx+2]):
        removed_idxs.append(idx+1)
    converted_df.drop(index=removed_idxs)

    # Group result
    results = [group[1] for group in converted_df.groupby([converted_df.direction_id, converted_df.point_id])]
    for group in results:
      direction = group.iloc[0]['direction_id']
      point_id = group.iloc[0]['point_id']

      group.to_csv(f"/content/main/normalized_lines/{line_id}_{direction}_{point_id}", index=False)

# Getting The Departure Time

We will cluster the normalized data based on the timestamp. This needs to be done in order to get estimated time when a bus/buses arrive at particular stop within the same time range. The time range need to be close enough to be considered as the same cluster. The departure time is calculated by picking the maximum timestamp from the same cluster.

In this approach we will use 4 algorithm to try cluster the data into the same group. We will use:<br/>
1) Ckmeans 1D DP<br/>
2) Dbscan 1D<br/>
3) Jenkins Natural Breaks<br/>
4) Kernel Density Estimation

We pick the algorithm based on how well the perform for one dimensional data, since most clustering algorithm is not optimized for handling such cases.

Steps to get the cluster: <br/>
1) Obtain the number of clusters needed. It is defined by the number of buses in pointID based on the schedule. The number of clusters is only needed for ckmeans and jenkins algorithm. <br/>
2) Cluster the data using the algorithm and append the label to the dataset <br/>

In [ ]:
# Removing schedules that are null
path = '/content/main/arrival_time/data/schedules/realFinal/central_lines_schedule_puncTrunc.csv'
df = pd.read_csv(path)

In [ ]:
df = df.loc[((df['avg_headway']!='None') | (df['estimated_departure_time']!='None'))]

In [ ]:
save_csv('arrival_time/data/schedules/realFinal/central_lines_schedule_puncTrunc_NoNone.csv', df)

In [ ]:
from numpy import array, linspace, sort
from sklearn.neighbors import KernelDensity
from matplotlib.pyplot import plot
from scipy.signal import argrelextrema
from jenkspy import JenksNaturalBreaks
from dbscan1d.core import DBSCAN1D
import ckwrap

# Extract all stops
# using stops.txt as it has all the stops
stops_df = pd.read_csv("/content/main/schedule/gtfs23Sept/stops.csv")
stops_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in stops_df["stop_id"]]
stops_df["stop_id"] = stops_df["stop_id"].astype(int)
all_stops = set([int(stop) for stop in stops_df["stop_id"]])
schedules_path = "/content/main/arrival_time/data/schedules/schedules_line/"

actual_df = pd.read_csv("/content/main/arrival_time/data/actual_stops.csv")
size = actual_df.shape[0]
actual_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in actual_df["stop_id"]]

actual_df["succession"] = actual_df["succession"].astype(str)
actual_df["numero_lig"] = actual_df["numero_lig"].astype(str)
actual_df["variante"] = actual_df["variante"].astype(str)

In [ ]:
faulty_stops = [] # Number of unprocessed stops
trip_headsign = 0 # index of trip headsign
stop_id = 2 # index of stop id
dtime = 8
n_trips = {}

# Save schedule to cache
lines = ['29']
for line_id in lines:
  try:
    n_trips_df = pd.read_csv(f"{schedules_path}schedule_{line_id}_rightTime.csv")
    n_trips_np = n_trips_df.to_numpy()
    n_trips[line_id] = n_trips_np

  except Exception:
    continue

In [ ]:
for filename in glob("/content/main/normalized_lines/*"):
    # print(filename)
    base_filename = os.path.basename(filename)
    try:
        line_id, direction, point_id = [int(v) for v in base_filename.split("_")]
        print(line_id, direction, point_id)
    except:
        continue

    if line != 29:
      continue

    print(filename)

    direction_descr = stops_df.loc[stops_df['stop_id']==direction].iloc[0]['stop_name']

    # checking for Technical stops
    if (point_id not in all_stops) or (direction not in all_stops):
        print('no')
        continue

    normalized_df = pd.read_csv(filename)
    normalized_df = normalized_df[(normalized_df["direction_id"] == direction)]
    normalized_df["converted_timestamp"] = normalized_df["converted_timestamp"].apply(lambda x: (pd.Timestamp(x)))

    arrival_time = normalized_df['converted_timestamp'].apply(lambda x: (x - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')).to_numpy()

    n_trips_np = n_trips[str(line_id)]

    start = normalized_df.iloc[0]["converted_timestamp"] + pd.Timedelta(hours=2)
    end = normalized_df.iloc[-1]["converted_timestamp"] + pd.Timedelta(hours=2)
    n = n_trips_np[( (n_trips_np[:,trip_headsign] == direction_descr) & (n_trips_np[:,stop_id] == point_id)
    & (n_trips_np[:,dtime] >= start) &  (n_trips_np[:,dtime] <= end) )].shape[0]

    # Check whether number of cluster is bigger than the data
    if arrival_time.shape[0] < n:
        faulty_stops.append((line_id, direction, point_id))
        continue

    # Check whether the schedule is exist or not
    if n==0:
      faulty_stops.append((line_id, direction, point_id))
      continue


    '''
        kmeans 1d
    '''
    try:
      first = arrival_time[0]
      data = np.array([(x-first) for x in arrival_time])
      result = ckwrap.ckmeans(data, n)
      normalized_df["kmeans"] = np.array(result.labels)
    except Exception as e:
      faulty_stops.append((line_id, direction, point_id))
      continue

    # Uncomment this line for doing clustering with other method

    #'''
    # Kernel Density estimation method
    # '''
    # first = arrival_time[0]
    # a = np.array([(x-first) for x in arrival_time])

    # a = np.array([1,1,1,2,3,4,10,12,16,40,45])
    # a = a.reshape(-1, 1)

    # kde = KernelDensity(kernel='gaussian', bandwidth=0.01).fit(a)

    # s = linspace(a[0],a[-1]+2000)
    # e = kde.score_samples(s.reshape(-1,1))
    # mi, ma = argrelextrema(e, np.less)[0], argrelextrema(e, np.greater)[0]

    # clusters = []

    # clusters.append(a[a < s[mi][0]])  # most left cluster

    # # print all middle cluster
    # for i_cluster in range(len(mi)-1):
    #     clusters.append(a[(a >= s[mi][i_cluster]) * (a <= s[mi][i_cluster+1])])

    # clusters.append(a[a >= s[mi][-1]])  # print most right cluster

    # '''
    #     dbsscan approach
    #     epsilon is 35 seconds
    # '''
    # dbs = DBSCAN1D(eps=35, min_samples=1)
    # labels = dbs.fit_predict(arrival_time)

    # '''
    #     Jenkins binning algorithm
    # '''
    # jnb = JenksNaturalBreaks(n)
    # jnb.fit(arrival_time)


    # normalized_df["dbscan"] = np.array(dbs.labels_)
    # normalized_df["jenkins"] = np.array(jnb.labels_)

    # kde_labels = []
    # for idx, cluster in enumerate(clusters):
    #     for x in cluster:
    #         kde_labels.append(idx)
    # normalized_df["KDE"] = np.array(kde_labels)

    save_csv(f'departure_time/cluster_{line_id}_{direction}_{point_id}.csv', normalized_df)

# Analyzing Departure Time

In [ ]:
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score

'''
Analyze the clustered data
'''
method = 'kmeans'

filename = '/home/sayyor/Desktop/BDMA_Studies/Data Mining/Project/Data/created_data/arrival_time_sample_cluster.csv'
normalized_df = pd.read_csv(filename)

# only if not CET+1
normalized_df.original_timestamp = pd.to_datetime(normalized_df['original_timestamp'])+pd.Timedelta('2h')
# df.converted_timestamp = df.converted_timestamp.apply(lambda x: parser.parse(x).timestamp()*int(TO_MS))

# getting measures for the clusters for each method
evals = {'ch':{},'db':{}, 'ss':{}}
normalized_df["converted_timestamp"] = normalized_df["converted_timestamp"].apply(lambda x: (pd.Timestamp(x)))
arrival_time = normalized_df['converted_timestamp'].apply(lambda x: (x - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')).to_numpy()
arrival_time = arrival_time.reshape(-1, 1)

for method in ['dbscan', 'kmeans', 'jenkins', 'KDE']:
  # measures[method]={'avr_in_variance':None, 'avr_out_gap':None}
  labels = normalized_df[method].to_numpy()
  res = calinski_harabasz_score(arrival_time, labels)
  evals['ch'][method] = round(res,2)

  res = davies_bouldin_score(arrival_time, labels)
  evals['db'][method] = round(res,2)

  res = silhouette_score(arrival_time, labels)
  evals['ss'][method] = round(res,2)

In [ ]:
from pandas.core.indexing import convert_from_missing_indexer_tuple
'''
Get Departure Time
'''

normalized_path = "/content/main/departure_time/*.csv"
result_path = "departure_time/results/"
already_done = [os.path.basename(filename) for filename in glob("/content/main/departure_time/results/*.csv")]
already_done = [f[:-4] for f in already_done]

method = "kmeans"

for clustered_f in glob(normalized_path):
  base_filename = os.path.basename(clustered_f)
  (line_id, direction_id, piont_id) = base_filename[8:-4].split("_")

  if line_id not in ['29']:
    continue

  if base_filename[8:-4] in already_done:
    continue

  print(line_id, direction_id, piont_id)

  df = pd.read_csv(clustered_f)
  departure_df = pd.DataFrame(columns=['departure_time'])

  for cluster in df[method].unique():
    records = df.loc[df[method]==cluster]
    departure_time = max(list(records.converted_timestamp))
    new_row = {
        'departure_time':departure_time
    }
    departure_df = departure_df.append(new_row, ignore_index=True)

  result_f = f'{result_path}{line_id}_{direction_id}_{piont_id}.csv'
  save_csv(result_f, departure_df)

In [ ]:
'''
Measuring Regularity
'''
regularity_faulty = []
headways_path = "/content/main/arrival_time/data/schedules/headways_line/"
for line_id in ['29']:
# for line_id in ['1','5', '29', '38', '63', '65', '66', '71', '89']:
  print('line',line_id)

  headways_file = f'{headways_path}headways_{line_id}_rightTime.csv'
  headways_np = np.genfromtxt(headways_file, delimiter=',', dtype=None, encoding=None)
  headways_np = headways_np[1:,:]
  save_path = "regularity/results/"

  headways_df = pd.read_csv(headways_file)

  stop_id_idx = get_column_idx("stop_id", headways_df)
  start_date_idx = get_column_idx("start_date", headways_df)
  end_date_idx = get_column_idx("end_date", headways_df)
  trip_headsign_idx = get_column_idx("trip_headsign", headways_df)
  interv_type_idx = get_column_idx("interval_type", headways_df)
  swt_idx = get_column_idx("swt", headways_df)
  awt_idx = get_column_idx("avg_headway", headways_df)
  init_len = headways_np.shape[1]

  start_datetime = [[datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")] for x in headways_np[:,start_date_idx]]
  headways_np = np.append(headways_np, start_datetime, axis=1)
  end_datetime = [[datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")] for x in headways_np[:,end_date_idx]]
  headways_np = np.append(headways_np, end_datetime, axis=1)
  start_date_idx = init_len
  end_date_idx = init_len+1

  ewt_arr = []
  ewt_arr.append(['stop_id', 'trip_headsign', 'interval_start', 'interval_end', 'avg_headway', 'swt','awt', 'ewt'])

  departure_path = "/content/main/departure_time/results/"
  departure_files = [f for f in glob(f"{departure_path}*.csv") if os.path.basename(f)[:-4].split("_")[0] == line_id]

  for real_time in departure_files:
    (line_id, direction_id, point_id) = (x for x in os.path.basename(real_time)[:-4].split("_"))
    print(line_id, direction_id, point_id)

    try:
      direction_descr = actual_df.loc[(actual_df['stop_id']==direction_id)].iloc[0]['descr_fr']
    except:
      regularity_faulty.append((line_id, direction_id, point_id))
      continue

    # getting in datetime format
    real_time_np = np.genfromtxt(real_time, delimiter=',', dtype=None, encoding=None)
    real_time_np = real_time_np[1:]
    real_time_np = np.char.strip(real_time_np, '"')
    real_time_np = np.array([[datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")] for x in real_time_np])

    # only the periods of the same pointID and directionID and evaluated by regularity
    periods = headways_np[( (headways_np[:,stop_id_idx]==point_id) & (headways_np[:, interv_type_idx] == 'regularity') & (headways_np[:, trip_headsign_idx] == direction_descr) )]
    print("n_periods", periods.shape[0])

    if periods.shape[0]==0:
      regularity_faulty.append((line_id, direction_id, point_id))

    # getting ewt for each period
    for period in periods:
      period_start = period[start_date_idx]
      period_end = period[end_date_idx]
      avg_headway = period[awt_idx]
      swt = float(period[swt_idx])

      # get departs for that period
      period_departs = real_time_np[((real_time_np >= period_start) & (real_time_np <= period_end))]

      # sorting by time
      period_departs = period_departs[period_departs.argsort()]

      # get headway for each pair of departure times
      n = period_departs.shape[0]
      headways = []
      if n == 0:
        continue
      elif n == 1:
        headway = (period_end - period_start).total_seconds()
        headway = headway/2
        headways.append(headway)
      else:
        for (first, second) in zip(range(n-1), range(1, n)):
            headway = period_departs[second]-period_departs[first]
            headways.append(headway)
        # convert to seconds each headway
        for i in range(len(headways)):
            headways[i] = headways[i].total_seconds()

      headways = np.array(headways)
      # calculate the metrics
      square_sum = np.sum(headways**2)
      sum_two = 2*np.sum(headways)
      awt = square_sum/sum_two
      ewt = awt-swt

      ewt_arr.append([point_id, direction_descr, period_start, period_end, avg_headway, swt, awt, ewt])

  ewt_arr = np.array(ewt_arr)
  save_np_csv(save_path+f"regularity_{line_id}.csv", ewt_arr)

In [ ]:
# combine results
line_id = '1'
path = '/content/main/arrival_time/data/schedules/realFinal/'
f = f'{path}schedule_{line_id}_rightTime.csv'
df = pd.read_csv(f)
n = df.shape[0]

lines = ['5', '29', '38', '63', '65', '66', '71', '89']
for line_id in lines:
  f = f'{path}schedule_{line_id}_rightTime.csv'
  df2 = pd.read_csv(f)
  n += df2.shape[0]
  df = df.append(df2)

In [ ]:
'''
remove regularity that is greater than threshold
'''
threshold = 1000
lines = ['1','5', '29', '38', '63', '65', '66', '71', '89']
regularity_res_path = '/content/main/regularity/results/'
out_path = 'regularity/until1000/'

for line_id in lines[1:]:
  print(line_id)
  regular_f = f'{regularity_res_path}regularity_{line_id}.csv'
  out_f = f'{out_path}regularity_{line_id}.csv'
  regular_df = pd.read_csv(regular_f)
  regular_df = regular_df.loc[(regular_df['ewt'] < 1000) & (regular_df['ewt'] > 0)]
  save_csv(out_f, regular_df)

5
29
38
63
65
66
71
89


In [ ]:
for line_id in ['1', '5', '38', '63', '65', '66', '71', '89', '29']:
  schedules_f = f'{schedules_path}schedule_{line_id}_rightTime.csv'
  schedules_df = pd.read_csv(schedules_f)
  print(line_id,'punctuality', schedules_df.loc[schedules_df['punctuality_index']!='None'].shape[0])
  print(line_id,'regularity', schedules_df.loc[schedules_df['ewt']!='None'].shape[0])
  print('\n')
  # schedules_df = schedules_df.drop(['estimated_departure_time', 'punctuality_index', 'avg_headway',	'swt','awt','ewt'], axis=1, errors='ignore')
  # new_f = schedules_f[14:]
  # save_csv(new_f, schedules_df)

In [ ]:
'''
Transfer awt and ewt to schedules_line files
Separate columns awt and ewt
Iterate through each line_id for schedules_line and regularity results
Each record that has regularity, the same stop_id, the same direction, time within the period
  should have the same awt and ewt
'''

schedules_path = '/content/main/arrival_time/data/schedules/schedules_line/'
regularity_res_path = '/content/main/regularity/until1000/'

# for line_id in ['1', '5', '29', '38', '63', '65', '66', '71'][1:]:
for line_id in ['89']:
  print(line_id)

  regular_f = f'{regularity_res_path}regularity_{line_id}.csv'
  schedules_f = f'{schedules_path}schedule_{line_id}_rightTime.csv'

  regular_np = np.genfromtxt(regular_f, delimiter=',', dtype=None, encoding=None)
  regular_np = np.char.strip(regular_np, '"')
  regular_np = regular_np[1:,:]
  reg_point_id, reg_direction_descr, reg_period_start, reg_period_end, reg_avg_headway, reg_swt, reg_awt, reg_ewt = range(8)

  period_start_np = [[datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")] for x in regular_np[:,reg_period_start]]
  regular_np = np.append(regular_np, period_start_np, axis=1)
  reg_period_start = 8
  period_end_np = [[datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")] for x in regular_np[:,reg_period_end]]
  regular_np = np.append(regular_np, period_end_np, axis=1)
  reg_period_end = 9

  schedules_np = np.genfromtxt(schedules_f, delimiter=',', dtype=None, encoding=None)
  schedules_np = np.char.strip(schedules_np, '"')
  schedules_np = schedules_np[1:,:]
  sch_direction_descr, _, sch_point_id, _, _, interv_type, _, _, sch_depart_time = range(9)

  # setting empty columns for avg_headway, swt, awt and ewt
  schedules_np = np.append(schedules_np,  np.full((schedules_np.shape[0],1), None), axis=1)
  schedules_np = np.append(schedules_np,  np.full((schedules_np.shape[0],1), None), axis=1)
  schedules_np = np.append(schedules_np,  np.full((schedules_np.shape[0],1), None), axis=1)
  schedules_np = np.append(schedules_np,  np.full((schedules_np.shape[0],1), None), axis=1)
  sch_avg_headway = 9
  sch_swt = 10
  sch_awt = 11
  sch_ewt = 12

  sch_depart_time_np = [[datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")] for x in schedules_np[:,sch_depart_time]]
  schedules_np = np.append(schedules_np, sch_depart_time_np, axis=1)
  sch_depart_time = 13

  i = 0
  for period in regular_np:
    i+=1
    if i % 100 == 0:
      print(i,regular_np.shape[0])
    match_indeces = np.where((
        (schedules_np[:,interv_type] == 'regularity') &
        (schedules_np[:,sch_point_id] == period[reg_point_id]) &
        (schedules_np[:,sch_direction_descr] == period[reg_direction_descr]) &
        (schedules_np[:,sch_depart_time] >= period[reg_period_start]) &
        (schedules_np[:,sch_depart_time] <= period[reg_period_end])
        ))

    for index in match_indeces[0]:
      schedules_np[index, sch_avg_headway] = period[reg_avg_headway]
      schedules_np[index, sch_swt] = period[reg_swt]
      schedules_np[index, sch_awt] = period[reg_awt]
      schedules_np[index, sch_ewt] = period[reg_ewt]

  schedules_np = np.delete(schedules_np, sch_depart_time, axis=1)

  columns = np.array([['trip_headsign', 'direction_id', 'stop_id', 'departure_time', 'headway', 'class', 'date', 'line', 'departure_date', 'avg_headway', 'swt', 'awt', 'ewt']])
  schedules_np = np.append(columns, schedules_np, axis=0)
  save_np_csv(schedules_f[14:], schedules_np)

89
100 314
200 314
300 314


In [ ]:
'''
Transfer estimated_departure_time and punctuality index to schedules_line files
Separate columns estimated_departure_time and punctuality_index
Iterate through each line_id for schedules_line and all files of the same line in punctuality/results
Each record that has punctuality, the same stop_id (in the filename),
  the same trip_headsign, the same scheduled_time
  should have the same estimated_departure_time and punctuality_index
'''

stops_df = pd.read_csv("/content/main/schedule/gtfs23Sept/stops.csv")
stops_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in stops_df["stop_id"]]
stops_df["stop_id"] = stops_df["stop_id"].astype(int)

schedules_path = '/content/main/arrival_time/data/schedules/schedules_line/'
regularity_res_path = '/content/main/punctuality/result/'

# for line_id in ['1', '5', '29', '38', '63', '65', '66', '89', '71'][1:]:
for line_id in ['89']:
  print(line_id)

  # schedules_f = f'{schedules_path}schedule_{line_id}_rightTime.csv'
  schedules_f = '/content/main/arrival_time/data/schedules/schedules_line/schedule_89_rightTime.csv'

  schedules_np = np.genfromtxt(schedules_f, delimiter=',', dtype=None, encoding=None)
  schedules_np = np.char.strip(schedules_np, '"')
  schedules_np = schedules_np[1:,:]
  sch_direction_descr, _, sch_point_id, _, _, interv_type, _, _, sch_depart_time, _,_,_,_ = range(13)

  # setting empty columns for avg_headway, swt, awt and ewt
  schedules_np = np.append(schedules_np,  np.full((schedules_np.shape[0],1), None), axis=1)
  schedules_np = np.append(schedules_np,  np.full((schedules_np.shape[0],1), None), axis=1)
  sch_est_depart_t = 13
  sch_punct_idx = 14

  sch_depart_time_np = [[pd.to_datetime(x)] for x in schedules_np[:,sch_depart_time]]
  schedules_np = np.append(schedules_np, sch_depart_time_np, axis=1)
  sch_depart_time = 15

  regular_fs = [f for f in glob(f"{regularity_res_path}*.csv") if os.path.basename(f).split("_")[0] == line_id]

  for regular_f in regular_fs:
    (_, direction_id, point_id, _) = os.path.basename(regular_f)[:-4].split("_")
    direction_descr = stops_df.loc[stops_df['stop_id']==int(direction_id)].iloc[0]['stop_name']
    print('f',regular_f)

    regular_np = np.genfromtxt(regular_f, delimiter=',', dtype=None, encoding=None)
    regular_np = np.char.strip(regular_np, '"')
    try:
      regular_np = regular_np[1:,:]
    except:
      continue
    reg_headsign, _, _, reg_sch_time, reg_punct_idx, reg_est_d_t, _, _ = range(8)
    # reg_headsign, _, _, _, reg_sch_time, reg_punct_idx, reg_est_d_t, _, _ = range(9)

    reg_sch_time_np = [[pd.to_datetime(x)] for x in regular_np[:,reg_sch_time]]
    regular_np = np.append(regular_np, reg_sch_time_np, axis=1)
    reg_sch_time = 8
    # reg_sch_time=9

    match_indeces = np.where((
      (schedules_np[:,interv_type] == 'punctuality') &
      (schedules_np[:,sch_point_id] == point_id) &
      (schedules_np[:,sch_direction_descr] == direction_descr)
      ))

    i = 0
    for period in regular_np:
      i+=1
      if i % 100 == 0:
        print(i,regular_np.shape[0])

      for m_idx in match_indeces[0]:
        if schedules_np[m_idx, sch_depart_time] == period[reg_sch_time]:
          schedules_np[m_idx, sch_est_depart_t] = period[reg_est_d_t]
          schedules_np[m_idx, sch_punct_idx] = period[reg_punct_idx]

  schedules_np = np.delete(schedules_np, sch_depart_time, axis=1)
  columns = np.array([['trip_headsign', 'direction_id', 'stop_id', 'departure_time', 'headway', 'class', 'date', 'line', 'departure_date','avg_headway', 'swt', 'awt', 'ewt', 'estimated_departure_time', 'punctuality_index']])
  schedules_np = np.append(columns, schedules_np, axis=0)
  save_np_csv(schedules_f[14:], schedules_np)

In [ ]:
'''Converting headway schedules to real time (25-->1 of the next day)'''
headways_path = "/content/main/arrival_time/data/schedules/headways_line/*.csv"

def get_column_idx(column_label, df):
  return [idx for idx, col in enumerate(df.columns) if col==column_label][0]

def get_real_date(year, month, day, hour, minute, second):
  date = datetime.datetime(year,month,day)
  if hour > 23:
    date += datetime.timedelta(days=1)
    hour = hour % 24
  date += datetime.timedelta(hours=hour, minutes=minute, seconds=second)
  return date

for headways_file in glob(headways_path):
  print(headways_file)

  headways_df = pd.read_csv(headways_file)
  headways_np = headways_df.to_numpy()
  init_len = headways_np.shape[1]

  start_t_idx = get_column_idx("interval_start", headways_df)
  end_t_idx = get_column_idx("interval_end", headways_df)
  date_idx = get_column_idx("date", headways_df)

  # splitting times into their components
  time_split = np.array([[int(x) for x in t.split(":")] for t in headways_np[:,start_t_idx]])
  headways_np = np.append(headways_np, time_split, axis=1)
  start_h_idx = init_len
  start_m_idx = init_len+1
  start_s_idx = init_len+2

  time_split = np.array([[int(x) for x in t.split(":")] for t in headways_np[:,end_t_idx]])
  headways_np = np.append(headways_np, time_split, axis=1)
  end_h_idx = init_len+3
  end_m_idx = init_len+4
  end_s_idx = init_len+5

  # splitting the date into its components
  headways_np[:, date_idx] = headways_np[:, date_idx].astype(str)
  date_split = np.array([[int(d[:4]), int(d[4:6]), int(d[6:])] for d in headways_np[:,date_idx]])
  date_y_idx = init_len+6
  date_m_idx = init_len+7
  date_d_idx = init_len+8
  headways_np = np.append(headways_np, date_split, axis=1)

  start_date = np.array([ [get_real_date(y,mo,d,h,m,s)] for [y,mo,d,h,m,s] in headways_np[:,[date_y_idx, date_m_idx, date_d_idx, start_h_idx, start_m_idx, start_s_idx]] ])
  end_date = np.array([ [get_real_date(y,mo,d,h,m,s)] for [y,mo,d,h,m,s] in headways_np[:,[date_y_idx, date_m_idx, date_d_idx, end_h_idx, end_m_idx, end_s_idx]] ])

  headways_np = np.append(headways_np, start_date, axis=1)
  headways_np = np.append(headways_np, end_date, axis=1)

  to_delete = [start_h_idx, start_m_idx, start_s_idx, end_h_idx, end_m_idx, end_s_idx, date_y_idx, date_m_idx, date_d_idx]
  headways_np = np.delete(headways_np, to_delete, 1)

  # adding the column to the beginning
  headers = list(headways_df.columns)
  headers.extend(['start_date', 'end_date'])
  headers = np.array([headers])
  headways_np = np.concatenate((headers, headways_np))

  save_np_csv(headways_file[14:-4]+'_rightTime.csv', headways_np)

In [ ]:
'''Converting schedules to real time (25-->1 of the next day)'''
headways_path = "/content/main/arrival_time/data/schedules/schedules_line/*.csv"

for headways_file in glob(headways_path):
  print(headways_file)
  date_idx = get_column_idx("date", headways_df)
  depart_t = get_column_idx("departure_time", headways_df)

  headways_df = pd.read_csv(headways_file)
  headways_np = headways_df.to_numpy()
  init_len = headways_np.shape[1]

  # splitting times into their components
  time_split = np.array([[int(x) for x in t.split(":")] for t in headways_np[:,depart_t]])
  headways_np = np.append(headways_np, time_split, axis=1)
  start_h_idx = init_len
  start_m_idx = init_len+1
  start_s_idx = init_len+2

  # splitting the date into its components
  headways_np[:, date_idx] = headways_np[:, date_idx].astype(str)
  date_split = np.array([[int(d[:4]), int(d[4:6]), int(d[6:])] for d in headways_np[:,date_idx]])
  date_y_idx = init_len+3
  date_m_idx = init_len+4
  date_d_idx = init_len+5
  headways_np = np.append(headways_np, date_split, axis=1)

  start_date = np.array([ [get_real_date(y,mo,d,h,m,s)] for [y,mo,d,h,m,s] in headways_np[:,[date_y_idx, date_m_idx, date_d_idx, start_h_idx, start_m_idx, start_s_idx]] ])

  headways_np = np.append(headways_np, start_date, axis=1)

  to_delete = [start_h_idx, start_m_idx, start_s_idx, date_y_idx, date_m_idx, date_d_idx]
  headways_np = np.delete(headways_np, to_delete, 1)

  # adding the column to the beginning
  headers = list(headways_df.columns)
  headers.extend(['departure_date'])
  headers = np.array([headers])
  headways_np = np.concatenate((headers, headways_np))

  save_np_csv(headways_file[14:-4]+'_rightTime.csv', headways_np)
# # periods = headways_np[( (headways_np[:,stop_id_idx]==point_id) & (headways_np[:, interv_type] == 'regularity') )]

In [ ]:
faulty_stops = [] # Number of unprocessed stops
trip_headsign = 0 # index of trip headsign
stop_id = 2 # index of stop id
dtime = 8
n_trips = {}

# Save schedule to cache
lines = ['1', '5', '2', '38', '63', '65', '66', '71', '89']
for line_id in lines:
  try:
    n_trips_df = pd.read_csv(f"{schedules_path}schedule_{line_id}_rightTime.csv")
    n_trips_np = n_trips_df.to_numpy()
    n_trips[line_id] = n_trips_np

  except Exception:
    continue

Analyzing Punctuality

In [ ]:
# Index definition
stop_type = 5
index = 10
punc_index = 9
sched_dtime = 8
real_dtime = 0
real_idx = 2
marked = 1
est_dtime = 11
est_dtime_marked = 12
punc_index_marked = 13
headway_idx = 6
stop = 2

for filename in glob("/content/main/departure_time/results/*.csv"):
  file_info = filename.split("/")[5].split("_")
  line_id, direction_id, stop_id = file_info[0], file_info[1], re.sub("\D", "", file_info[2].strip())

  if "29" != line_id:
    continue

  real_data = pd.read_csv(filename)
  start = pd.to_datetime(real_data.iloc[0]['departure_time']) + pd.Timedelta(hours=2)
  end = pd.to_datetime(real_data.iloc[-1]['departure_time']) + pd.Timedelta(hours=2)

  direction_descr = stops_df[stops_df['stop_id'] == int(direction_id)].iloc[0]['stop_name']

  n_trips_np = n_trips[line_id]
  n_trips_np[:,sched_dtime] = pd.to_datetime(n_trips_np[:,sched_dtime])
  n_trips_np = n_trips_np[(n_trips_np[:,trip_headsign] == direction_descr) & (n_trips_np[:,stop] == int(stop_id)) & (n_trips_np[:, sched_dtime] >= start) & (n_trips_np[:, sched_dtime] <= end)]

  # Add initial punctuality and indexes for processing
  est_dtime_arr = [None for i in range(n_trips_np.shape[0])]
  est_dtime_marked_arr = [None for i in range(n_trips_np.shape[0])]
  punc_idx_marked_arr = [None for i in range(n_trips_np.shape[0])]
  punctuality = [None for i in range(n_trips_np.shape[0])]
  idxs = [i for i in range(n_trips_np.shape[0])]
  n_trips_np = np.c_[n_trips_np, punctuality, idxs, est_dtime_arr, est_dtime_marked_arr, punc_idx_marked_arr]

  # Add variables to real data for punctuality processing
  real_data['departure_time'] = real_data['departure_time'].apply(lambda x: pd.to_datetime(x) + pd.Timedelta(hours=2))
  real_data['marked'] = [False for i in range(real_data.shape[0])]
  real_data['index'] = [i for i in range(real_data.shape[0])]
  real_data = real_data.to_numpy()

  # Filter punctuality only trips
  trips = n_trips_np[n_trips_np[:, stop_type] == 'punctuality']

  for idx, trip in enumerate(trips):
    time = trip[sched_dtime] - pd.Timedelta(minutes=1)
    flt_rt = real_data[real_data[:,real_dtime] >= time]
    headway = (n_trips_np[idx+1][headway_idx]) if idx+1 < len(n_trips_np) - 1 else 1
    punc_idx = 'NaN'

    for rt in flt_rt:
      ori_idx = rt[real_idx]
      punc_idx = (rt[real_dtime] - time).seconds / headway
      n_trips_np[trip[index]][est_dtime] = rt[real_dtime]
      break
    n_trips_np[trip[index]][punc_index] = punc_idx

    punc_idx_marked = "NaN"
    for rt in flt_rt:
      ori_idx = rt[real_idx]
      if real_data[ori_idx][marked]:
        continue
      punc_idx_marked = (rt[real_dtime] - time).seconds / headway
      real_data[ori_idx][marked] = True
      n_trips_np[trip[index]][est_dtime_marked] = rt[real_dtime]
      break
    n_trips_np[trip[index]][punc_index_marked] = punc_idx_marked

  schedule = pd.DataFrame(n_trips_np)
  schedule = schedule[schedule[stop_type] == 'punctuality'].drop(columns=[1,2,6,7,index])
  save_csv(f"punctuality/result/{line_id}_{direction_id}_{stop_id}_punctuality.csv", schedule)


In [ ]:
punctualities = {}
for filename in glob("/content/main/punctuality/result/*.csv"):
    file_info = filename.split("/")[5].split("_")
    line_id, direction_id, stop_id = file_info[0], file_info[1], file_info[2]
    punctuality = punctualities.setdefault(line_id, None)

    punctuality_df = pd.read_csv(filename)
    punctuality_df["stop_id"] = stop_id

    if punctuality is None:
      punctuality = punctuality_df
    else:
      punctuality = pd.concat([punctuality, punctuality_df], ignore_index=True)

    punctualities[line_id] = punctuality
    print(file_info)

for line_id, df in punctualities.items():
  save_csv(f"punctuality/combined/{line_id}_punctuality.csv", df)

In [ ]:
csv = "/content/main/arrival_time/data/schedules/schedules_line/schedule_71_rightTime_update.csv"
combined_df = pd.read_csv().to_numpy()
# for filename in glob("/content/main/punctuality/combined/*.csv"):
punctuality_df = pd.read_csv("/content/main/punctuality/combined/*.csv").to_numpy()



In [ ]:
# Analysis